In [1]:
class KF2(object): #new
    def Configure(self, Q_accel, R_altitude, h, v):
        self.Q_accel = Q_accel
        self.R_altitude = R_altitude
        
        self.h = h
        self.v = v
       
        self.P = [
            [1.0, 0.0],
            [0.0, 1.0]
        ]
        
    def propagate(self, acceleration, dt):
        _dtdt = dt * dt;
        self.h = self.h + self.v * dt + 0.5 * acceleration * _dtdt;
        self.v = self.v + acceleration * dt;

        _Q_accel_dtdt = self.Q_accel * _dtdt;
        
        self.P[0][0] = self.P[0][0] + (self.P[1][0] + self.P[0][1] + (self.P[1][1] + 0.25*_Q_accel_dtdt) * dt) * dt;
        self.P[0][1] = self.P[0][1] + (self.P[1][1] + 0.5*_Q_accel_dtdt) * dt;
        self.P[1][0] = self.P[1][0] + (self.P[1][1] + 0.5*_Q_accel_dtdt) * dt;
        self.P[1][1] = self.P[1][1] + _Q_accel_dtdt;
        
    def update(self, altitude):
        y = altitude - self.h
        Sinv = 1.0 / (self.P[0][0] + self.R_altitude)
        K = [self.P[0][0] * Sinv, self.P[1][0] * Sinv]
        self.h += K[0] * y;
        self.v += K[1] * y;
        
        self.P[0][0] = self.P[0][0] - K[0] * self.P[0][0];
        self.P[0][1] = self.P[0][1] - K[0] * self.P[0][1];
        self.P[1][0] = self.P[1][0] - (K[1] * self.P[0][0]);
        self.P[1][1] = self.P[1][1] - (K[1] * self.P[0][1]);
        
    def Update(self, z_abs, accel, dt = 0.01):
        self.propagate(accel, dt)
        self.update(z_abs)
        
        return self.h, self.v

    def Update1(self, z_abs, accel, dt = 0.01):
        self.update(z_abs)
        self.propagate(accel, dt)
        
        return self.h, self.v


In [2]:
import threading
import serial

class read_thread(threading.Thread):
    def __init__(self):
        threading.Thread.__init__(self)
        self.running = True
        self.start()
        self.values = []
        self.lock = threading.Lock()
        
    def stop(self):
        self.running = False
        
    def get_values(self):
        with self.lock:
            val = self.values
            self.values = []
            return val
        
    def run(self):
        while (self.running):
            soc = serial.Serial("/dev/ttyUSB0", 921600)

            try:
                values = soc.read_until().decode('ascii').split()[0].split(";")
                if values[0] == "S" and len(values) == 5:
                    raw_alt, raw_acc, filt_alt, filt_vario = map(float, values[-4:])
                
                with self.lock:
                    self.values.append([raw_alt, raw_acc, filt_alt, filt_vario])
            except Exception as error:
                print(error, values)

In [ ]:
%matplotlib tk

import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
import numpy as np

plt.ion()

fig, ax = plt.subplots(2, 1, sharex=True)
ax1, ax2 = ax
plt.subplots_adjust(left=0.03, bottom=0.15, top = 0.98, right = 0.99, hspace = 0)

x = np.arange(5, step = 0.01)

tot_len = len(x)
empty = [0] * tot_len

p_raw_alts, = ax1.plot(x, empty, "o", label="raw alt", markersize=0.5)
p_filt_alts, = ax1.plot(x, empty, label="filt alt")

p_raw_acc, = ax2.plot(x, empty, "y-", label = "raw acc")
p_filt_vario, = ax2.plot(x, empty, "b-", label = "filt vario")

p_filt_acc, = ax2.plot(x, empty, "r-", label = "filt acc")
ax2.plot(x, empty, "k-", label = "zero")


ax1.legend(loc='best', shadow=True)    
ax2.legend(loc='best', shadow=True)    
fig.show()

raw_alts = empty
raw_accs = empty
filt_alt = empty
filt_vario = empty

filt_accs = empty
filt_acc = 0

data = read_thread()
while True:
    for row in data.get_values():
        raw_alts = (raw_alts + [row[0]])[-tot_len:]
        raw_accs = (raw_accs + [row[1]/9.81])[-tot_len:]
        filt_alt = (filt_alt + [row[2]])[-tot_len:]
        filt_vario = (filt_vario + [row[3]])[-tot_len:]
        
        filt_acc += ((row[1]/9.81) - filt_acc) / 10.0
        filt_accs = (filt_accs + [filt_acc])[-tot_len:]
       
    a1_min = min(min(raw_alts), min(filt_alt))
    a1_max = max(max(raw_alts), max(filt_alt))

    a2_min = min(min(raw_accs), min(filt_vario))
    a2_max = max(max(raw_accs), max(filt_vario))
    
    ax1.set_ylim(a1_min, a1_max)
    ax2.set_ylim(a2_min, a2_max)
    
    p_raw_alts.set_ydata(raw_alts)
    p_filt_alts.set_ydata(filt_alt)

    p_raw_acc.set_ydata(raw_accs)
    p_filt_vario.set_ydata(filt_vario)
    
    p_filt_acc.set_ydata(filt_accs)
    
    
    
    fig.canvas.draw_idle()
    plt.pause(0.01)
    

/usr/lib/python3/dist-packages/matplotlib/axes/_base.py:3285: UserWarning: Attempting to set identical bottom==top results
in singular transformations; automatically expanding.
bottom=0, top=0
  'bottom=%s, top=%s') % (bottom, top))


local variable 'raw_alt' referenced before assignment ['\x00']
list index out of range ['S', '107.9897', '0.0000', '107.9302', '0.0122']
